In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 设置请求头，模拟浏览器访问
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
    "Cookie": "_lxsdk_cuid=193b544a72fc8-026d202573c394-26011851-144000-193b544a72fc8; _lxsdk=193b544a72fc8-026d202573c394-26011851-144000-193b544a72fc8; _hc.v=65c3183f-14b4-7d2c-f46d-88a102276353.1733912996; s_ViewType=10; ua=dpuser_3514064540; ctu=5f9373f1a224b576a5e42830c308791e18489f3daf5c11028df005d64e30d897; cityid=98; fspop=test; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1738237847; HMACCOUNT=C2A9454786D9FDA0; WEBDFPID=uuvwz3z02v21506wy331v8x002z4yzyu806y455wz479795861zw4zzu-1738324397486-1733913006752GAGAESCfd79fef3d01d5e9aadc18ccd4d0c95073349; qruuid=3925bdfd-9a2a-4889-8e9d-fedcbd7ee6cd; dplet=c8386e43e25d0e573efc85e16aca3283; dper=0202f9201b66bc08b350edc319a4e79085385273c0869259c6d1c20c94e197f5683e4cd47a31dee4f83dc1f64edfb46f3823dcb4fcf160187cba0000000084260000554d59cb4b5065980715aaaea152764698aeb77fbea91b3960a02744b64f07feda310a9d28b1d01402253053c3042c91; ll=7fd06e815b796be3df069dec7836c3df; cy=9; cye=chongqing; __CACHE@is_login=true; __CACHE@referer=https://www.dianping.com/search/keyword/9/0_%E9%A4%90%E5%8E%85; logan_custom_report=; logan_session_token=9a4c725il7uil5t3fzri; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1738313420; _lxsdk_s=194bb8da6c1-ef8-041-ed%7C%7C42"
}

# 目标URL模板
base_url = "https://www.dianping.com/chongqing/ch10/p"

# 爬取的页数
total_pages = 50

# 存储数据的列表
data_list = []

# 解析评分
def parse_rating(stars):
    if "star_50" in stars:
        return 5.0
    elif "star_45" in stars:
        return 4.5
    elif "star_40" in stars:
        return 4.0
    elif "star_35" in stars:
        return 3.5
    elif "star_30" in stars:
        return 3.0
    elif "star_25" in stars:
        return 2.5
    elif "star_20" in stars:
        return 2.0
    elif "star_15" in stars:
        return 1.5
    elif "star_10" in stars:
        return 1.0
    elif "star_05" in stars:
        return 0.5
    else:
        return 0.0

# 遍历多个页面
for page in range(1, total_pages + 1):
    url = f"{base_url}{page}"
    print(f"正在爬取：{url}")
    
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"第 {page} 页请求失败，状态码：{response.status_code}")
        continue
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # 解析所有店铺信息
    shop_list = soup.find_all("li", class_="")

    for shop in shop_list:
        try:
            # 店铺名称
            name_tag = shop.find("h4")
            name = name_tag.text.strip() if name_tag else "未知"

            # 详情链接
            link_tag = shop.find("a", attrs={"data-click-name": "shop_title_click"})
            link = link_tag["href"] if link_tag else "未知"

            # 图片链接
            img_tag = shop.find("img")
            img_url = img_tag["data-src"] if img_tag and "data-src" in img_tag.attrs else "未知"

            # 评分
            star_tag = shop.find("span", class_="star")
            star_class = star_tag["class"] if star_tag else []
            rating = parse_rating(" ".join(star_class))

            # 评价数量
            review_tag = shop.find("a", class_="review-num")
            reviews = review_tag.b.text.strip() if review_tag else "未知"

            # 人均消费
            price_tag = shop.find("a", class_="mean-price")
            price = price_tag.b.text.strip().replace("￥", "") if price_tag else "未知"

            # 地理位置
            location_tags = shop.find_all("span", class_="tag")
            location = " | ".join(tag.text.strip() for tag in location_tags) if location_tags else "未知"

            # 推荐菜品
            recommend_tag = shop.find("div", class_="recommend")
            if recommend_tag:
                dishes = [a.text.strip() for a in recommend_tag.find_all("a")]
                recommended_dishes = "，".join(dishes)
            else:
                recommended_dishes = "无推荐菜"

            # 存储数据
            data_list.append([name, link, img_url, rating, reviews, price, location, recommended_dishes])
        
        except Exception as e:
            print(f"解析店铺信息时发生错误：{e}")

    # 避免被封，增加随机延迟
    time.sleep(3)

# 存储为 DataFrame
df = pd.DataFrame(data_list, columns=["店铺名称", "链接", "图片链接", "评分", "评价数量", "人均消费", "地理位置", "推荐菜品"])

# 保存为 CSV 文件，确保中文显示
df.to_csv("重庆美食餐厅.csv", index=False, encoding="utf-8-sig")

print("数据爬取完成，已保存为重庆美食餐厅.csv")

In [ ]:
import requests
import csv
import re
import time
import random
from tqdm import tqdm
from fake_useragent import UserAgent
import pandas as pd

# 随机生成 User-Agent
def get_random_useragent():
    ua = UserAgent()
    return ua.random

# 用于更新 headers
def update_headers(cookie):
    headers = {
        "User-Agent": get_random_useragent(),  # 使用随机UA避免被封
        "Cookie": cookie
    }
    return headers

# 解析评论数据
def parse_reviews(contents):
    review_list = []

    # 正则表达式提取所需信息
    user_id_pattern = re.compile(r'<a class="name".*?data-user-id="(?P<user_id>\d+)"', re.S)
    overall_score_pattern = re.compile(r'<span class="sml-rank-stars sml-str(?P<score>\d+).*?</span>', re.S)
    taste_score_pattern = re.compile(r'口味：(?P<taste>\d+\.\d+)', re.S)
    environment_score_pattern = re.compile(r'环境：(?P<environment>\d+\.\d+)', re.S)
    service_score_pattern = re.compile(r'服务：(?P<service>\d+\.\d+)', re.S)
    comment_pattern = re.compile(r'<div class="review-words.*?>(?P<comment>.*?)</div>', re.S)
    liked_dish_pattern = re.compile(r'喜欢的菜：.*?<a.*?>(?P<dish>.*?)</a>', re.S)
    time_pattern = re.compile(r'<span class="time">\s*(?P<time>.*?)\s*</span>', re.S)

    user_ids = user_id_pattern.findall(contents)
    overall_scores = overall_score_pattern.findall(contents)
    taste_scores = taste_score_pattern.findall(contents)
    environment_scores = environment_score_pattern.findall(contents)
    service_scores = service_score_pattern.findall(contents)
    comments = comment_pattern.findall(contents)
    liked_dishes = liked_dish_pattern.findall(contents)
    times = time_pattern.findall(contents)

    # 处理评论内容
    processed_comments = []
    for comment in comments:
        processed_comment = re.sub(r"&#x0A;|&#x20;|<img.*?>", "", comment).strip()
        processed_comments.append(processed_comment)

    # 补全缺失数据
    max_len = max(len(user_ids), len(overall_scores), len(taste_scores), len(environment_scores),
                  len(service_scores), len(processed_comments), len(times))

    user_ids += ["匿名"] * (max_len - len(user_ids))
    overall_scores += ["0"] * (max_len - len(overall_scores))
    taste_scores += ["0"] * (max_len - len(taste_scores))
    environment_scores += ["0"] * (max_len - len(environment_scores))
    service_scores += ["0"] * (max_len - len(service_scores))
    processed_comments += ["无"] * (max_len - len(processed_comments))
    times += ["未知时间"] * (max_len - len(times))
    liked_dishes += ["无"] * (max_len - len(liked_dishes))

    for i in range(max_len):
        review_list.append([
            user_ids[i],
            overall_scores[i],
            taste_scores[i],
            environment_scores[i],
            service_scores[i],
            processed_comments[i],
            liked_dishes[i],
            times[i]
        ])

    return review_list

# 清洗评论内容的函数
def clean_comment_v3(comment):
    # Remove spaces first
    comment = comment.replace(" ", "").strip()
    # Remove the entire <a> tag that contains '收起评价' and its content
    comment = re.sub(r'<a[^>]*>收起评价.*?</a>', '', comment)
    # Also remove any remaining tags that might have been missed
    comment = re.sub(r'<[^>]+>', '', comment)
    comment = comment.replace("收起评价", "").strip()
    return comment

# 爬取评论
def scrape_comments(shop_id, min_page, max_page, headers):
    filename = f"FOODBOWL超级碗(中信泰富广场店).csv"

    # 打开 CSV 文件
    with open(filename, mode="w", newline="", encoding="utf-8-sig") as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(["用户ID", "总体评分", "口味评分", "环境评分", "服务评分", "评论内容", "喜欢的菜", "评论时间"])

        for i in tqdm(range(min_page, max_page + 1)):
            url = f"https://www.dianping.com/shop/{shop_id}/review_all/p{i}"
            response = requests.get(url, headers=headers)
            response.encoding = "utf-8-sig"

            if response.status_code == 200:
                reviews = parse_reviews(response.text)
                for review in reviews:
                    csv_writer.writerow(review)
                print(f"第 {i} 页爬取完成")
            else:
                print(f"第 {i} 页请求失败，状态码：{response.status_code}")

            # 随机休眠，避免被封
            time.sleep(random.uniform(3, 6))

    print(f"所有评论爬取完成，已保存至 {filename}")

if __name__ == '__main__':
    # 用户输入信息
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'  # 您可以直接粘贴 UA
    cookie = 'cy=1; cityid=1; cye=shanghai; GSI=#sl_zdjlwxbdsb@211011l#rf_tsmbyrk@220309b#rf_syztwlb@210720a#rf_syjgwlb@220829a; _lxsdk_cuid=193b544a72fc8-026d202573c394-26011851-144000-193b544a72fc8; _lxsdk=193b544a72fc8-026d202573c394-26011851-144000-193b544a72fc8; _hc.v=65c3183f-14b4-7d2c-f46d-88a102276353.1733912996; s_ViewType=10; ctu=5f9373f1a224b576a5e42830c308791e18489f3daf5c11028df005d64e30d897; cityid=98; fspop=test; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1738237847; HMACCOUNT=C2A9454786D9FDA0; __CACHE@is_login=true; logan_custom_report=; cityname=%E9%95%87%E6%B1%9F; cy=1; cye=shanghai; WEBDFPID=uuvwz3z02v21506wy331v8x002z4yzyu806y455wz479795861zw4zzu-1738586097610-1733913006752GAGAESCfd79fef3d01d5e9aadc18ccd4d0c95073349; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; Hm_lvt_6bc30b61e75e1c8b9c509994370bd6e9=1738499775; Hm_lpvt_6bc30b61e75e1c8b9c509994370bd6e9=1738499789; __CACHE@referer=https://www.dianping.com/search/keyword/1/0_%E8%B6%85%E7%BA%A7%E7%A2%97; logan_session_token=027lzmsxi29nqci16sp9; qruuid=f18980b0-858a-46be-8ac7-49d87bee42f2; dplet=191ae46df36173e0d774b3932768cb70; dper=0202541f74b9e701a749b2ce6a18a2dc7252ba55d26eec122087e5889347327e98436c5f19d8bee4ee05eb6866afc6ec3872e13425c20b450a0b0000000084260000197e371587874273c434977baaef554b85e5333ccf63221d470c8dc9b3417a75bc825b9f093cfd4dadf81623aa0d20b4; ll=7fd06e815b796be3df069dec7836c3df; ua=dpuser_3514064540; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1738509485; _lxsdk_s=194c6a6a762-858-10d-972%7C%7C3957'  # 直接粘贴您的 Cookie
    shop_id = 'l2yPNFK0pLypmHAQ'
    min_page = int(input("请输入最小页码："))
    max_page = int(input("请输入最大页码："))

    headers = update_headers(cookie)
    scrape_comments(shop_id, min_page, max_page, headers)
    # 加载数据
    data = pd.read_csv('FOODBOWL超级碗(中信泰富广场店).csv')

    # 清洗评论内容
    data['评论内容'] = data['评论内容'].apply(clean_comment_v3)

    # 保存清洗后的数据到新的CSV文件
    cleaned_file_path = '15.FOODBOWL超级碗(中信泰富广场店).csv'
    data.to_csv(cleaned_file_path, index=False, encoding='utf-8-sig')
